In [7]:
#Load Libraries
from datetime import date
from sportsreference.nba.teams import Teams
import pyodbc
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

In [8]:
#for development in azure notebooks (databricks uses spark behind the scenes and does not require this step)
sc = SparkContext('local')
spark = SparkSession(sc)

In [9]:
#Create Dataframe
DimTeam = pd.DataFrame()

#Load Team name and Abbreviation into Dataframe
for x in range(2000,2020):
    for team in Teams(x):
      DimTeam = DimTeam.append({'Name':team.name,'Abbreviation':team.abbreviation},ignore_index='true')
    x=x+1
 

In [10]:
#load into spark dataframe
DimTeamSparkDF = spark.createDataFrame(DimTeam)
#DimTeamSparkDF.createOrReplaceTempView("DimTeam")
columnsToHash = [x for x in DimTeamSparkDF.columns if str(x) not in ['Abbreviation']]
DimTeamSparkDF = DimTeamSparkDF.withColumn("RowValueHash", sha2(concat_ws("|", *columnsToHash), 256))
DimTeamSparkDF.createOrReplaceTempView("DimTeam")

In [11]:
#select * from df
spark.sql(
    'Select distinct * \
    from DimTeam \
    Order By name desc' 
    ).show()

+------------+--------------------+--------------------+
|Abbreviation|                Name|        RowValueHash|
+------------+--------------------+--------------------+
|         WAS|  Washington Wizards|8f17f23fb75312b56...|
|         VAN| Vancouver Grizzlies|d7f90b05d059686aa...|
|         UTA|           Utah Jazz|7c902bb2e27276e38...|
|         TOR|     Toronto Raptors|417f4fff4625e3c2d...|
|         SEA| Seattle SuperSonics|5f3dde5c5e7d7926b...|
|         SAS|   San Antonio Spurs|e89f51275352e48d9...|
|         SAC|    Sacramento Kings|04e36c744934d2dbb...|
|         POR|Portland Trail Bl...|0054c2679f7753beb...|
|         PHO|        Phoenix Suns|9bf9a5fd18b1063b4...|
|         PHI|  Philadelphia 76ers|edf03ad3c346f7941...|
|         ORL|       Orlando Magic|cc72cd00eb9505787...|
|         OKC|Oklahoma City Thu...|d8ec6878976ac9c12...|
|         NYK|     New York Knicks|6a36e386117ede8ea...|
|         NOK|New Orleans/Oklah...|ffdb6339526d63087...|
|         NOP|New Orleans Pelic